# Librairies


In [ ]:
import os
import cv2
import numpy as np
import pickle
from base64 import b64decode
from IPython.display import display, Javascript
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from google.colab.output import eval_js

# Création du dataset

In [ ]:
# ================== #
# Variables globales #
# ================== #

# Création d'un répertoire data
if not os.path.exists("data"):
    os.mkdir("data")

nb_images_collectees = 10 # Nombre d'image qu'on veut collecter pour chacun des admis et des non admis
donnees_visage = [] # Liste vide pour enregistrer les visages

# =============================== #
# Capture des data: nom et images #
# =============================== #

# Définition de la fonction pour capturer une photo
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
        const div = document.createElement('div');
        const capture = document.createElement('button');  // bouton pour capture d'image
        capture.textContent = 'Capture';
        div.appendChild(capture);
        // permet l'affichage du flux vidéo de notre caméra en temps réel
        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
        // l'exécution du script s'interromp jusqu'à ce qu'un clic pour la capture soit effectué
        await new Promise((resolve) => capture.onclick = resolve);
        //  capture de l'image, arrêt de la caméra, mise en format jpeg de l'image
        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getVideoTracks()[0].stop();
        div.remove();
        return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    # On exécute le script JavaScript et on réceptionne l'image capturée sous forme de chaîne de données base64
    data = eval_js('takePhoto({})'.format(quality))
    # on décode la chaîne base64 pour obtenir les données de l'image en binaire
    binary = b64decode(data.split(',')[1])
    # Sauvegarde des données binaires dans un fichier image
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# on charge le modèl qui permet de détecter les visage
cascade_visage = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

nom = input("Entrez votre nom: ")  # pour saisir le nom
for i in range(nb_images_collectees):
    filename = take_photo()  # Capture la photo
    image = cv2.imread(filename)
    gris = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    coordonnees_visage = cascade_visage.detectMultiScale(gris, 1.3, 5)

    for (x, y, w, h) in coordonnees_visage:
        visage = image[y:y+h, x:x+w]
        visage_redimensionne = cv2.resize(visage, (50, 50))
        donnees_visage.append(visage_redimensionne)
        break  # on sauvegarde que la 1er photo

# on convertit les données des visages en un tableau numpy
donnees_visage = np.array(donnees_visage, dtype='float32')
donnees_visage /= 255.0 # on normmalise ces données

# Enregistrement des données des noms et des visages
noms_path = "data/noms.pkl"
if os.path.exists(noms_path):
    with open(noms_path, 'rb') as file:
        noms = pickle.load(file)
else:
    noms = []

noms += [nom] * nb_images_collectees
with open(noms_path, 'wb') as file:
    pickle.dump(noms, file)

visages_path = "data/visages.pkl"
if os.path.exists(visages_path):
    with open(visages_path, 'rb') as file:
        visages_existants = pickle.load(file)
    visages = np.concatenate((visages_existants, donnees_visage), axis=0)
else:
    visages = donnees_visage

with open(visages_path, 'wb') as file:
    pickle.dump(visages, file)

Entrez votre nom: beyonce


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Algorithme

In [ ]:
# Encodage des noms en vecteurs binaires
encoder = LabelBinarizer()
noms_encoded = encoder.fit_transform(noms)

# Diviser les visages et les noms en ensemnle d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(donnees_visage, noms_encoded, test_size=0.25, random_state=42)

# on redimensionne les données
X_train = X_train.reshape(-1, 50, 50, 3)
X_test = X_test.reshape(-1, 50, 50, 3)

In [ ]:
# Modèle de réseau de neurones convulotionnels
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 3)), # 1ère couche avec 32 filtre de taille 3x3, activation relu et l'entrée avec des photos 50x50
    MaxPooling2D(2, 2), # 1ère couche de pooling
    Conv2D(64, (3, 3), activation='relu'),  # 2ème couche avec 64 filtre
    MaxPooling2D(2, 2), # 2ème couche de pooling
    Flatten(),# Aplatit en un vecteur 1D
    Dense(64, activation='relu'), # Couche dense avec 64 unités
    Dropout(0.5), # Couche de Dropout
    Dense(len(encoder.classes_), activation='softmax') # Couche de sortie
])

# on compile notre modèle
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Entraînement du modèle avec un nombre d'époques égal à 10
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Enregistrement du modèle et de l'encodeur
model.save('modele_reconnaissance_faciale.keras')
with open('encodeur_classes.pkl', 'wb') as f:
    pickle.dump(encoder, f)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


1/1 [==============================] - 2s 2s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 74ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 80ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 65ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 78ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 68ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 66ms/step - loss: 0.0000e+00 -

# Exécution

In [ ]:
# Chargement du modèle et de l'encodeur
modele = load_model('modele_reconnaissance_faciale.keras')
with open('encodeur_classes.pkl', 'rb') as f:
    encoder = pickle.load(f)
filename = take_photo()  # Capture une nouvelle image
image = cv2.imread(filename)
image = cv2.resize(image, (50, 50))  # Redimensionne la photo prise
image = image.astype('float32') / 255.0  # Normalisation
image = np.expand_dims(image, axis=0)

# Prédiction
prediction = modele.predict(image)
classe_predite = encoder.classes_[np.argmax(prediction)]

print(f"Classe prédite : {classe_predite}")

<IPython.core.display.Javascript object>

1/1 [==============================] - 0s 106ms/step
Classe prédite : beyonce
